In [1]:
##Logan Lauton

##importing packages needed for scrape
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import time
import requests
import warnings

##function designed to scrape all of the injuries reported in NBA history 
def single(tot):
    ##number that determines where the start is, or in other words what page of the table the url is 
    ##as it increments by 25, page 1 => start=0 page 2 => start=25 page 3 => start=50 etc
    num = 0
    ##supress warnining for using df.append rather than pd.concat
    warnings.filterwarnings('ignore', message='The frame.append method is deprecated')
    while tot != 0:
        if num == 0:
            url = f'https://www.prosportstransactions.com/basketball/Search/SearchResults.php?Player=&Team=&BeginDate=&EndDate=&ILChkBx=yes&Submit=Search&start={num}'
            response = requests.get(url)
            html = response.content
            soup = BeautifulSoup(html, 'lxml')
            table = soup.find('table', attrs={'class': 'datatable center'})
            ##here I create the df that will be populated throughout the scrape
            df = pd.read_html(str(table))[0]
            num = num + 25           ##increment to 25 to move onto page 2
            tot = tot - 1            ##reduce tot to see how many pages are left to visit
            time.sleep(2.38)         ##rate limit to get the total corpus(n=1507) in 1 hour (≈ 25.21 requests/minute)
        else:
            url = f'https://www.prosportstransactions.com/basketball/Search/SearchResults.php?Player=&Team=&BeginDate=&EndDate=&ILChkBx=yes&Submit=Search&start={num}'
            response = requests.get(url)
            html = response.content
            soup = BeautifulSoup(html, 'lxml')
            table = soup.find('table', attrs={'class': 'datatable center'})
            ##here I append to the df that was created 
            df = df.append(pd.read_html(str(table))[0])
            num = num + 25           ##increment the start to go to the next page
            tot = tot - 1            ##reduce the total to determine number of remaining pages
            time.sleep(2.38)         ##rate limit to get the total corpus(n=1507) in 1 hour (≈ 25.21 requests/minute)

    df.columns = ['Date','Team','Acquired','Relinquished','Notes']
    df = df[df['Date']!='Date']
    df.reset_index(drop=True, inplace=True)
    df.Acquired = df.Acquired.str.replace('•','') # remove • in front of player's name
    df.Relinquished = df.Relinquished.str.replace('•','') # remove • in front of player's name
    df['Date'] = pd.to_datetime(df['Date'].str.strip(), format='%Y-%m-%d')
    return df

In [2]:
df = single(1507)

In [3]:
##exporting df as csv
df.to_csv('NBA Player Injury Stats(1951 - 2023).csv', index = True)